In [ ]:
#vlp = pMHC
#tcr = TCR

import pandas as pd

# Step 1: Load the VLP dataframe (adjust this based on input)
vlp_df = pd.read_csv('RAPTR-VLP.csv', header=0, index_col=0)  # Assuming the VLPs are row indexes
tcr_df = pd.read_csv('RAPTR-TCR.csv', header=0, index_col=0)  # Assuming the TCRs are row indexes

# Step 2: Transpose the dataframes to get barcodes as a column
vlp_transposed_df = vlp_df.T.reset_index()
vlp_transposed_df.columns.values[0] = 'Barcode'  # Rename the first column to 'Barcode'
tcr_transposed_df = tcr_df.T.reset_index()
tcr_transposed_df.columns.values[0] = 'Barcode'  # Rename the first column to 'Barcode'

# Step 3: Remove the accidental '.1' from the barcode column
vlp_transposed_df['Barcode'] = vlp_transposed_df['Barcode'].str.replace('.1', '', regex=False)
tcr_transposed_df['Barcode'] = tcr_transposed_df['Barcode'].str.replace('.1', '', regex=False)

# Process TCR dataframe

In [ ]:
import matplotlib.pyplot as plt

# Step 1: Sum UMIs per barcode (row)
umi_counts = tcr_transposed_df.iloc[:, 1:].sum(axis=1)  # Exclude the 'Barcode' column

# Step 2: Plot histogram
plt.figure(figsize=(10, 6))
plt.hist(umi_counts, bins=50, color='blue', edgecolor='black', alpha=0.7)
plt.xlabel('Total UMIs per Cell Barcode')
plt.ylabel('Number of Cells')
plt.title('Distribution of UMIs per Cell Barcode')
plt.yscale('log')  # Log scale for better visibility if there are large variations
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

## Filter cells with < 5 UMIs

In [ ]:
# Step 1: Calculate total UMIs per barcode (row)
tcr_transposed_df['Total_UMIs'] = tcr_transposed_df.iloc[:, 1:].sum(axis=1)  # Exclude 'Barcode' column

# Debugging: Print summary of UMIs before filtering
print("Before filtering:")
print(tcr_transposed_df['Total_UMIs'].describe())

# Step 2: Filter out rows with total UMIs < 5
filtered_tcr_df = tcr_transposed_df[tcr_transposed_df['Total_UMIs'] >= 5].copy()  # Ensure a new DataFrame is created

# Debugging: Print summary after filtering
print("\nAfter filtering:")
print(filtered_tcr_df['Total_UMIs'].describe())

# Step 3: Drop the 'Total_UMIs' column if no longer needed
filtered_tcr_df = filtered_tcr_df.drop(columns=['Total_UMIs'])

In [ ]:
## Examine distribution of TCR UMIs for each cell barcode

import matplotlib.pyplot as plt

# Normalize each row by its sum (total UMIs per cell)
normalized_tcr_df = filtered_tcr_df.set_index("Barcode")  # Ensure Barcode is the index
normalized_tcr_df = normalized_tcr_df.div(normalized_tcr_df.sum(axis=1), axis=0)  # Row-wise normalization

# Get the maximum proportion for each cell (row)
max_proportions = normalized_tcr_df.max(axis=1)

## Filter cells where the proportion of each TCR is < x%

In [ ]:
# Define the threshold
threshold = 0.6

# Identify barcodes where the maximum proportion is >= x%
high_confidence_barcodes = normalized_tcr_df.index[normalized_tcr_df.max(axis=1) >= threshold]

# Filter the original UMI count dataframe based on these barcodes
high_confidence_tcr_df = filtered_tcr_df.loc[filtered_tcr_df["Barcode"].isin(high_confidence_barcodes)]

# Print the number of remaining cells
print(f"Number of cells remaining after filtering: {high_confidence_tcr_df.shape[0]}")


In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

# Step 1: Convert the UMI counts dataframe to a sparse matrix
# Convert the dataframe without the 'Barcode' column
umi_matrix = csr_matrix(high_confidence_tcr_df.iloc[:, 1:].values)

# Step 2: Initialize an empty list to store the results
tcr_identity_data = []

# Step 3: Loop through each row (cell barcode) to get the top TCR identities
for i, row in enumerate(umi_matrix):
    barcode = high_confidence_tcr_df.iloc[i, 0]  # Get the barcode for this row
    
    # Get the sorted indices of the UMIs in descending order (excluding 0s)
    sorted_indices = row.toarray().flatten().argsort()[::-1]
    
    # Get the top 2 TCR identities and their UMI counts
    top_tcr_identity_1 = high_confidence_tcr_df.columns[sorted_indices[0] + 1]  # +1 because the first column is 'Barcode'
    top_umi_1 = row[0, sorted_indices[0]]
    
    if len(sorted_indices) > 1 and top_umi_1 > 0:  # Check if there is a second TCR identity
        top_tcr_identity_2 = high_confidence_tcr_df.columns[sorted_indices[1] + 1]
        top_umi_2 = row[0, sorted_indices[1]]
    else:
        top_tcr_identity_2 = None
        top_umi_2 = None
    
    # Step 4: Append to the result list
    tcr_identity_data.append([barcode, top_tcr_identity_1, top_umi_1, top_tcr_identity_2, top_umi_2])

# Step 5: Convert the list to a dataframe
result_df = pd.DataFrame(tcr_identity_data, columns=['Barcode', 'Top TCR Identity 1', 'UMIs 1', 'Top TCR Identity 2', 'UMIs 2'])

In [ ]:
# Step 1: Create a new dataframe with only the top TCR and top VLP identities
top_tcr_df = result_df[['Barcode', 'Top TCR Identity 1', 'UMIs 1']]

# Display the new dataframe
top_tcr_df

# Process pMHC dataframe

In [ ]:
vlp_transposed_df

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

# Step 1: Calculate total UMIs per barcode (row) for the VLP dataframe
vlp_transposed_df['Total_UMIs'] = vlp_transposed_df.iloc[:, 1:].sum(axis=1)  # Exclude 'Barcode' column

# Step 2: Filter out rows with total UMIs < 5 (you can adjust the threshold if needed)
filtered_vlp_df = vlp_transposed_df[vlp_transposed_df['Total_UMIs'] >= 5].copy()

# Step 3: Drop the 'Total_UMIs' column as it's no longer needed
filtered_vlp_df = filtered_vlp_df.drop(columns=['Total_UMIs'])

filtered_vlp_df

In [ ]:
## Examine distribution of VLP UMIs for each cell barcode

import matplotlib.pyplot as plt

# Normalize each row by its sum (total UMIs per cell)
normalized_vlp_df = filtered_vlp_df.set_index("Barcode")  # Ensure Barcode is the index
normalized_vlp_df = normalized_vlp_df.div(normalized_vlp_df.sum(axis=1), axis=0)  # Row-wise normalization

# Get the maximum proportion for each cell (row)
max_vlp_proportions = normalized_vlp_df.max(axis=1)

# Plot histogram
plt.figure(figsize=(8, 6))
plt.hist(max_vlp_proportions, bins=50, color='royalblue', edgecolor='black', alpha=0.7)
plt.xlabel("Maximum Proportion of a Single pMHC Identity")
plt.ylabel("Number of Cells")
plt.title("Distribution of Maximum pMHC Proportion per Cell")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# Optional: Filter cells where proportion of VLPs is < x%

# Define the threshold
threshold = 0

# Identify barcodes where the maximum proportion is >= x%
high_confidence_vlp_barcodes = normalized_vlp_df.index[normalized_vlp_df.max(axis=1) >= threshold]

# Filter the original UMI count dataframe based on these barcodes
high_confidence_vlp_df = filtered_vlp_df.loc[filtered_vlp_df["Barcode"].isin(high_confidence_vlp_barcodes)]

# Print the number of remaining cells
print(f"Number of cells remaining after filtering: {high_confidence_vlp_df.shape[0]}")

In [ ]:
# Initialize a list to store results for the new dataframe
vlp_identity_data = []

# Loop through each row (cell barcode) to get the top 2 VLP identities
for _, row in high_confidence_vlp_df.iterrows():
    barcode = row['Barcode']
    
    # Get the VLP identities (excluding the 'Barcode' column) and sort by UMIs
    vlp_counts = row[1:].sort_values(ascending=False)
    
    # Step 6: Get the top 2 VLP identities and their associated UMI counts
    top_vlp_identity_1 = vlp_counts.index[0]
    top_umi_1 = vlp_counts.iloc[0]
    
    if len(vlp_counts) > 1:  # Check if there is a second VLP identity
        top_vlp_identity_2 = vlp_counts.index[1]
        top_umi_2 = vlp_counts.iloc[1]
    else:  # If only one VLP identity exists, use NaN for the second
        top_vlp_identity_2 = None
        top_umi_2 = None
    
    # Append the data to the list
    vlp_identity_data.append([barcode, top_vlp_identity_1, top_umi_1, top_vlp_identity_2, top_umi_2])

# Create a new dataframe with the collected data
result_vlp_df = pd.DataFrame(vlp_identity_data, columns=['Barcode', 'Top VLP Identity 1', 'UMIs 1', 'Top VLP Identity 2', 'UMIs 2'])

# Display the result
result_vlp_df

# Merge dataframes

In [ ]:
# Step 1: Merge the TCR and VLP dataframes on 'Barcode'
merged_df = pd.merge(result_df, result_vlp_df, on='Barcode', how='inner')

# Display the merged dataframe
merged_df

In [ ]:
# Step 1: Create a new dataframe with only the top TCR and top VLP identities
top_tcr_vlp_df = merged_df[['Barcode', 'Top TCR Identity 1', 'UMIs 1_x', 'Top VLP Identity 1', 'UMIs 1_y']]

# Display the new dataframe
top_tcr_vlp_df

# From merged dataframe, create pivot tables to generate figures

In [ ]:
import pandas as pd

# Step 1: Rename columns
top_tcr_vlp_df = top_tcr_vlp_df.rename(columns={
    'cell barcode': 'Barcode',
    'Top TCR Identity 1': 'TCR',
    'UMIs 1_x': 'UMIs_TCR',
    'Top VLP Identity 1': 'VLP',
    'UMIs 1_y': 'UMIs_VLP'
})

# Step 2: Create a dataframe with unique TCRs as rows and all VLP identities as columns
heatmap_df = top_tcr_vlp_df.pivot_table(index='TCR', columns='VLP', values='Barcode', aggfunc='count', fill_value=0)

# Step 3: Ensure all 101 VLP identities from vlp_transposed_df are present
vlp_identities = vlp_transposed_df.columns[1:-1]  # Exclude 'Barcode' column
heatmap_df = heatmap_df.reindex(columns=vlp_identities, fill_value=0)

In [ ]:
# Filter TCRs with less than x cells

cell_threshold = 5

heatmap_unfiltered_df = heatmap_df

# Step 1: Compute total number of cells for each TCR identity (sum of each row)
heatmap_df['Total_Cells'] = heatmap_df.sum(axis=1)

# Step 2: Filter out TCR identities where Total_Cells < x
heatmap_df = heatmap_df[heatmap_df['Total_Cells'] >= cell_threshold]

# Step 3: Sort by total cells in descending order
heatmap_df = heatmap_df.sort_values(by='Total_Cells', ascending=False)

# Step 4: Drop the "Total_Cells" column after sorting (optional, if not needed in the heatmap)
heatmap_df = heatmap_df.drop(columns=['Total_Cells'])

In [ ]:
heatmap_unfiltered_df

In [ ]:
# Step 1: Extract the last column
tcr_cell_counts = heatmap_unfiltered_df.iloc[:, -1]

# Step 2: Create a new DataFrame with the TCR identity (from index) and Total Cells
tcr_cell_counts = pd.DataFrame({
    'TCR Identity': heatmap_unfiltered_df.index,  # TCR identity from the index
    'Total Cells': tcr_cell_counts  # The last column (Total Cells)
})

# Step 3: Sort in descending order by Total Cells
tcr_cell_counts = tcr_cell_counts.sort_values(by='Total Cells', ascending=False)

# Step 4: Display the first few rows
print(tcr_cell_counts)


In [ ]:
# Create a single-color colormap with your color choice
from matplotlib.colors import LinearSegmentedColormap

# Define a custom single color colormap (example with #bf4542)
single_color_cmap = LinearSegmentedColormap.from_list("single_red", ['white','#8a3330'])#641e1a

# Step 1: Filter out TCRs with less than 5 cells
tcr_cell_counts_filtered = tcr_cell_counts[tcr_cell_counts['Total Cells'] >= 5]

# Step 2: Convert the filtered dataframe into a heatmap-friendly format (set TCR Identity as index)
heatmap_tcr_data = tcr_cell_counts_filtered.set_index('TCR Identity')

# Step 3: Create the heatmap
plt.figure(figsize=(1, 10))  # Adjust the figure size as needed
sns.heatmap(heatmap_tcr_data, cmap=single_color_cmap, annot=False, fmt='d', cbar_kws={'label': 'Total Cells'}, linewidths=0.5, linecolor="gray")

# Customize the plot
plt.title('Heatmap of Total Cells per TCR Identity', fontsize=16, fontname='Helvetica')
plt.xlabel('TCR Identity', fontsize=14, fontname='Helvetica')
plt.ylabel('Total Cells', fontsize=14, fontname='Helvetica')

# Set the font to Helvetica for all text
plt.xticks(fontname='Helvetica')
#plt.yticks(fontname='Helvetica')

plt.savefig("Heatmap_cell_number.pdf", format="pdf", bbox_inches='tight')

plt.show()

# Heatmap based on number of cells for each TCR-pMHC pair

In [ ]:
# Step 1: Normalize each row by the total number of cells for that TCR identity
heatmap_normalized_df = heatmap_df.div(heatmap_df.sum(axis=1), axis=0)

In [ ]:
heatmap_normalized_df.to_csv('heatmap_cells>5.csv')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(30, 10))  # Adjust figure size
sns.heatmap(heatmap_normalized_df, cmap="Blues", linewidths=0.5, linecolor="gray") #YlGnBu and BuPu good color scheme

# Step 3: Customize the plot
plt.xlabel("VLP Identity")
plt.ylabel("TCR Identity")
plt.title("Normalized Heatmap of TCR-VLP Pairings (Fraction of Total Cells for Each TCR)")

# Step 4: Show the heatmap
plt.show()

# Heatmap based on UMIs for each TCR-pMHC pair

In [ ]:
# Merge all

# Step 1: Merge the TCR and VLP dataframes on 'Barcode'
merged_umi_df = pd.merge(top_tcr_df, filtered_vlp_df, on='Barcode', how='inner')

# Display the merged dataframe
merged_umi_df

In [ ]:
# Pivot the dataframe to create the heatmap_umi_df
heatmap_umi_df = merged_umi_df.pivot_table(
    index="Top TCR Identity 1",  # TCR identity as rows
    values=merged_umi_df.columns[3:],  # Use all VLP identity columns
    aggfunc="sum",  # Sum UMIs across TCR-VLP pairs
    fill_value=0  # Replace NaN with 0
)

# Display the result
heatmap_umi_df

In [ ]:
import pandas as pd

heatmap_umi_df['Total_UMIs'] = heatmap_umi_df.sum(axis=1)

# Step 1: Reorder the rows in descending order based on the existing 'Total_UMIs' column
heatmap_umi_df = heatmap_umi_df.sort_values(by='Total_UMIs', ascending=False)

# Step 2: Keep only the top 32 rows based on total UMIs
heatmap_umi_df_top = heatmap_umi_df.head(30)

# Step 3: Normalize all values by the corresponding 'Total_UMIs' column
heatmap_umi_df_normalized = heatmap_umi_df_top.div(heatmap_umi_df_top['Total_UMIs'], axis=0)

# Step 4: Drop the 'Total_UMIs' column as it is no longer needed
heatmap_umi_df_normalized = heatmap_umi_df_normalized.drop(columns=['Total_UMIs'], errors='ignore')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Step 1: Create a heatmap
plt.figure(figsize=(30,10))  # Adjust figure size as needed
sns.heatmap(heatmap_umi_df_normalized, cmap="Blues", linewidths=0.5, linecolor="gray", cbar_kws={'label': 'Normalized UMIs'})

# Step 2: Customize the plot
plt.xlabel("VLP Identity")
plt.ylabel("TCR Identity")
plt.title("Normalized Heatmap of VLP UMIs per TCR Identity")

# Step 3: Show the heatmap
plt.savefig("UMI_heatmap.pdf",format='pdf',bbox_inches='tight')
plt.show()

# Bar plots for individual TCRs

In [ ]:
pmhc_list_df = pd.read_csv("pMHC_list.csv",index_col=None,header=None)
pmhc_list_df

In [ ]:
# Based on cells

import matplotlib.pyplot as plt

# Step 1: Filter the collapsed dataframe for individual TCR identities'
tcr_identity = 'TCR-2922'  # TCR identity you're interested in
tcr_data = heatmap_df.loc[tcr_identity]

# Ensure tcr_data is a DataFrame
if isinstance(tcr_data, pd.Series):
    tcr_data = tcr_data.to_frame()

# Replace index with the second column of pmhc_list_df
#tcr_data.index = pmhc_list_df.iloc[:, 1].values
#tcr_data.index = range(1,102)

# Sort the index alphabetically
#tcr_data = tcr_data.sort_index()

# Display the sorted DataFrame
#tcr_data


In [ ]:
import matplotlib.pyplot as plt  
import seaborn as sns  

# Ensure tcr_data is a Series
if isinstance(tcr_data, pd.DataFrame):
    tcr_data = tcr_data.iloc[:, 0]  # Select the first column if it's a DataFrame

# Step 2: Plot the bar chart  
plt.figure(figsize=(18, 5))  # Adjust figure size as needed
sns.barplot(x=tcr_data.index, y=tcr_data.values.flatten(), color="#005292")  # Use a consistent color  

# Step 3: Customize the plot  
plt.xlabel("Peptide", fontsize=14, fontname="Helvetica")  
plt.ylabel("# cells", fontsize=14, fontname="Helvetica")  
plt.title(f"Cell Count for {tcr_identity}", fontsize=16, fontname="Helvetica")  
plt.xticks(rotation=90, fontsize=10, fontname="Helvetica")  # Rotate x labels  
plt.yticks(fontsize=15, fontname="Helvetica")  

# Step 4: Show the plot
#tcr_data.to_csv("1403_cells.csv")
#plt.savefig("Individual_TCR_1432_cells_heatmap.pdf",format='pdf',bbox_inches='tight')
plt.show()



# pMHC-TCR pairs

In [ ]:
heatmap_unfiltered_df

In [ ]:
import pandas as pd

# Step 1: Drop the final column of heatmap_df
heatmap_unfiltered_df = heatmap_unfiltered_df.iloc[:, :-1]  # Keep all rows, but drop the last column

In [ ]:
# Step 2: Sum across the rows to generate a total number of cells for each TCR identity
heatmap_unfiltered_df['Total Cells'] = heatmap_unfiltered_df.sum(axis=1)

# Step 3: Find the maximum number of cells for each TCR identity (excluding 'Total Cells' column)
max_vlp_cells = heatmap_unfiltered_df.iloc[:, :-1].max(axis=1)

# Step 4: Identify all VLP identities that have this maximum cell count
max_vlp_identity = heatmap_unfiltered_df.iloc[:, :-1].apply(lambda row: row[row == row.max()].index.tolist(), axis=1)

# Combine results into a DataFrame
summary_df = pd.DataFrame({
    'TCR Identity': heatmap_unfiltered_df.index,
    'Total Cells': heatmap_unfiltered_df['Total Cells'],
    'pMHC Identity with Most Cells': max_vlp_identity,
    'Number of Cells for Most Common pMHC': max_vlp_cells
})

In [ ]:
summary_df = summary_df.sort_values(by='Total Cells', ascending=False)
summary_df

In [ ]:
summary_df.to_csv('TCR-pMHC_pairs_based_on_cells.csv')

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Modify the TCR identity labels by removing the "TCR-" prefix
summary_df["TCR Identity"] = summary_df["TCR Identity"].str.replace("TCR-", "", regex=False)

# Convert pMHC lists to strings for unique identification
summary_df["pMHC Identity with Most Cells"] = summary_df["pMHC Identity with Most Cells"].apply(lambda x: ", ".join(x) if isinstance(x, list) else str(x))

# Define color palette
color_palette = ["#bf4542", "#A1C9E1", "#F0ABA9", "#E4C455", "#005292",  
                 "#947427", "#5C8F4F", "#924E7D", "#76A5AF", "#D76A8A", "#F0A45D", "#FF5733"]

light_grey = "#989EB0"  # Light grey for wedges with <5 cells

# Compute total cells per pMHC identity (for sorting)
total_counts = summary_df.groupby("pMHC Identity with Most Cells")["Total Cells"].sum()

# Sort identities by total fraction (largest first)
sorted_pMHCs = total_counts.sort_values(ascending=False).index.tolist()

# Create a mapping from unique pMHC identity combinations to colors
color_map = {pMHC: color_palette[i % len(color_palette)] for i, pMHC in enumerate(sorted_pMHCs)}

# Assign colors based on the mapped pMHC identities
summary_df["Color"] = [
    color_map[pMHC] if total >= 5 else light_grey  
    for pMHC, total in zip(summary_df["pMHC Identity with Most Cells"], summary_df["Total Cells"])
]

# Sort the dataframe to group identities together and order by total fraction
summary_df["Sort Order"] = summary_df["pMHC Identity with Most Cells"].map(lambda x: sorted_pMHCs.index(x))
summary_df = summary_df.sort_values(by=["Sort Order", "Total Cells"], ascending=[True, False])

# Step 1: Handle "Other" category (combine small fractions)
threshold = 5  # Adjust this threshold as needed
other_df = summary_df[summary_df["Total Cells"] < threshold]
other_sum = other_df["Total Cells"].sum()

# Step 2: Remove small fractions from the main dataframe
summary_df = summary_df[summary_df["Total Cells"] >= threshold]

# Step 3: Create and store the "Other" category as a separate row, if it exists
if other_sum > 0:
    other_df = pd.DataFrame({
        "pMHC Identity with Most Cells": ["Other"],
        "Total Cells": [other_sum],
        "Color": ["grey"],  # You can choose a color for "Other"
        "Sort Order": [len(sorted_pMHCs)]  # Put "Other" at the end
    })
    
    # Append the "Other" row back to the summary dataframe
    summary_df = pd.concat([summary_df, other_df], ignore_index=True)

# Step 4: Plot the pie chart with "Other" as a single block wedge and a black outline
plt.figure(figsize=(10, 10))

plt.pie(
    summary_df["Total Cells"], 
    labels=None,  # No labels
    startangle=140, 
    colors=summary_df["Color"],  
    wedgeprops={"edgecolor": "black", "linewidth": 0.5}  # Black borders
)

# Create a legend for pMHC identities, excluding grey wedges (<5 cells)
filtered_pMHCs = [pMHC for pMHC in sorted_pMHCs if total_counts[pMHC] >= 5]

legend_handles = [
    plt.Line2D([0], [0], marker='s', color='w', markersize=10, markerfacecolor=color_map[p])
    for p in filtered_pMHCs
]

# Add "Other" to the legend as well
if other_sum > 0:
    legend_handles.append(plt.Line2D([0], [0], marker='s', color='w', markersize=10, markerfacecolor="grey"))
    filtered_pMHCs.append("Other")

plt.legend(legend_handles, filtered_pMHCs, title="pMHC Identity", loc="upper center", fontsize=10, bbox_to_anchor=(0.5, -0.1),
           ncol=5, frameon=False, columnspacing=1.0, handleheight=1.5, handlelength=1.5, labelspacing=0.5, title_fontsize='large')

# Ensure circular aspect ratio and save
plt.axis('equal')
plt.savefig("TCR_identity_pie_chart.pdf", format='pdf', bbox_inches='tight')
plt.show()